In [1]:
from pathlib import Path

from vitessce import (
    AnnDataWrapper,
    ImageOmeZarrWrapper,
    ObsSegmentationsOmeZarrWrapper,
    VitessceConfig,
)
from vitessce import (
    CoordinationLevel as CL,
)

In [2]:
fov = "TONIC_TMA5_R1C2"
vitessce_fovs = Path("../data/vitessce_ingestable")

In [3]:
vc = VitessceConfig(schema_version="1.0.16", name="Vitessce spatialBeta testing")

In [4]:
dataset = (
    vc.add_dataset(name=fov)
    .add_object(
        AnnDataWrapper(
            adata_path=vitessce_fovs / fov / "whole_cell_table.zarr",
            obs_feature_matrix_path="X",
            obs_locations_path="obsm/spatial",
            obs_labels_path="obs/label",
            obs_set_paths=[["obs/cell_cluster_broad", "obs/cell_cluster", "obs/cell_meta_cluster"]],
            obs_set_names=["Cell Type"],
            coordination_values={
                "fileUid": "adata-table",
                "obsType": "Cell Segmentation",
                "featureType": "gene",
                "featureValueType": "expression",
            },
        )
    )
    .add_object(
        ImageOmeZarrWrapper(img_path=vitessce_fovs / fov / "image.ome.zarr", coordination_values={"fileUid": "image"})
    )
    .add_object(
        ObsSegmentationsOmeZarrWrapper(
            img_path=vitessce_fovs / fov / "cell_segmentation.ome.zarr",
            obs_types_from_channel_names=True,
            coordination_values={"fileUid": "segmentation"},
        )
    )
)
spatial_view = vc.add_view("spatialBeta", dataset=dataset)
lc_view = vc.add_view("layerControllerBeta", dataset=dataset)
obs_sets_view = vc.add_view("obsSets", dataset=dataset)

selectionScope, colorScope = vc.add_coordination("obsSetSelection", "obsSetColor")

obs_sets_view.use_coordination(selectionScope, colorScope)

imageLayerCL = CL(
    [
        {
            "fileUid": "image",
            "spatialLayerOpacity": 1,
            "spatialLayerVisible": True,
            "photometricInterpretation": "BlackIsZero",
            "spatialTargetResolution": None,
            "imageChannel": CL(
                [
                    {
                        "spatialTargetC": 0,
                        "spatialChannelColor": [255, 0, 0],
                        "spatialChannelVisible": True,
                        "spatialChannelOpacity": 1,
                        "spatialChannelWindow": [0, 255],
                    },
                ]
            ),
        }
    ]
)
segmentationLayerCL = CL(
    [
        {
            "fileUid": "segmentation",
            "spatialLayerVisible": True,
            "spatialLayerOpacity": 1,
            "segmentationChannel": CL(
                [
                    {
                        "obsType": "Cell Segmentation",
                        "spatialTargetC": 0,
                        "spatialChannelColor": [255, 255, 255],
                        "spatialChannelOpacity": 1.0,
                        "featureType": "gene",
                        "featureValueType": "expression",
                        "spatialChannelVisible": True,
                        "spatialSegmentationFilled": False,
                        "spatialSegmentationStrokeWidth": 0.01,
                        "obsHighlight": None,
                        "obsSelection": selectionScope,
                        "obsSetColor": colorScope,
                    },
                    {
                        "obsType": "Nuclear Segmentation",
                        "spatialTargetC": 1,
                        "spatialChannelColor": [255, 255, 255],
                        "spatialChannelOpacity": 0.2,
                        "featureType": "gene",
                        "featureValueType": "expression",
                        "spatialChannelVisible": True,
                        "spatialSegmentationFilled": False,
                        "spatialSegmentationStrokeWidth": 0.01,
                        "obsHighlight": None,
                        "obsSelection": selectionScope,
                        "obsSetColor": colorScope,
                    },
                ]
            ),
        }
    ]
)

vc.link_views_by_dict(
    views=[spatial_view, lc_view, obs_sets_view],
    input_val={"imageLayer": imageLayerCL, "segmentationLayer": segmentationLayerCL},
)

In [5]:
import pprint

In [6]:
pprint.pprint(vc.to_dict(base_url="http://localhost:3000"))

{'coordinationSpace': {'dataset': {'A': 'A'},
                       'featureType': {'A': 'gene', 'B': 'gene'},
                       'featureValueType': {'A': 'expression',
                                            'B': 'expression'},
                       'fileUid': {'A': 'image', 'B': 'segmentation'},
                       'imageChannel': {'A': '__dummy__'},
                       'imageLayer': {'A': '__dummy__'},
                       'metaCoordinationScopes': {'A': {'imageLayer': ['A'],
                                                        'segmentationLayer': ['A']}},
                       'metaCoordinationScopesBy': {'A': {'imageChannel': {'spatialChannelColor': {'A': 'A'},
                                                                           'spatialChannelOpacity': {'A': 'A'},
                                                                           'spatialChannelVisible': {'A': 'A'},
                                                                           's

In [7]:
vc.layout(spatial_view | lc_view | obs_sets_view)

In [8]:
vc.web_app()

'http://vitessce.io/#?theme=light&url=data:,%7B%22version%22%3A+%221.0.16%22%2C+%22name%22%3A+%22Vitessce+spatialBeta+testing%22%2C+%22description%22%3A+%22%22%2C+%22datasets%22%3A+%5B%7B%22uid%22%3A+%22A%22%2C+%22name%22%3A+%22TONIC_TMA5_R1C2%22%2C+%22files%22%3A+%5B%7B%22fileType%22%3A+%22anndata.zarr%22%2C+%22url%22%3A+%22http%3A%2F%2Flocalhost%3A8000%2FA%2F0%2F1512c4d5-3029-49e6-a0dd-31082598c3dc.adata.zarr%22%2C+%22options%22%3A+%7B%22obsLocations%22%3A+%7B%22path%22%3A+%22obsm%2Fspatial%22%7D%2C+%22obsSets%22%3A+%5B%7B%22name%22%3A+%22Cell+Type%22%2C+%22path%22%3A+%5B%22obs%2Fcell_cluster_broad%22%2C+%22obs%2Fcell_cluster%22%2C+%22obs%2Fcell_meta_cluster%22%5D%7D%5D%2C+%22obsFeatureMatrix%22%3A+%7B%22path%22%3A+%22X%22%7D%2C+%22obsLabels%22%3A+%5B%7B%22path%22%3A+%22obs%2Flabel%22%2C+%22obsLabelsType%22%3A+%22label%22%7D%5D%7D%2C+%22coordinationValues%22%3A+%7B%22fileUid%22%3A+%22adata-table%22%2C+%22obsType%22%3A+%22Cell+Segmentation%22%2C+%22featureType%22%3A+%22gene%22%2C+%22f

In [ ]:
vc